In [ ]:
import sys
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import *


In [ ]:
spark = SparkSession\
    .builder\
    .appName("SparkETL")\
    .getOrCreate()

##Read and split data ##

In [ ]:
from pyspark.sql.types import *

In [ ]:
peopleSchema = StructType([
    #define rowid
    StructField('row_id',IntegerType(), True),
    #define id
    StructField('id',IntegerType(), True),
    #define name
    StructField('name',StringType(), True),
    #define sex
    StructField('sex',StringType(), True),
    #define DOB
    StructField('dob',DateType(), True),
])

In [ ]:
df_people = spark.read.csv("s3://BUCKET_NAME/input/people.csv", header=True, schema=peopleSchema)

In [ ]:
df_people.show(5)

In [ ]:
df_people.printSchema()

In [ ]:
df_parq = df_people.write.mode("overwrite").parquet("s3://BUCKET_NAME/output/people.parquet")

In [ ]:
#remove duplicates
df_people_sub = df_people.select('name','sex','dob')
df_people_nodup = df_people_sub.dropDuplicates()

In [ ]:
print("There were {} rows before removing duplicates, and {} rows after removing duplicates".format(df_people_sub.count(), df_people_nodup.count()))

In [ ]:
#filter
df_people_female = df_people_nodup.filter(df_people_nodup.sex == "female")
df_people_male =df_people_nodup.filter(df_people_nodup.sex == "male")

In [ ]:
print("There are {} rows in the people_df_female DataFrame and {} rows in the people_df_male DataFrame".format(df_people_female.count(), df_people_male.count()))